![Coleridge Initiative](../support/images/CI_horizontal.png)

# Text Analysis
---

## Table of Contents

- [Introduction](#Introduction)

    - [Learning Outcomes](#Learning-Outcomes)
    - [Glossary of Terms](#Glossary-of-Terms)

- [Setup](#Setup)
- [Data Orientation](#Data-Orientation)

    - [Load the Data](#Load-the-Data)
    - [Explore the Data](#Explore-the-Data)
    - [How many facilities and types of facilities are in this dataset?](#How-many-facilities-and-types-of-facilities-are-in-this-dataset?)

- [Topic Modeling](#Topic-Modeling)

    - [Preparing Text Data for NLP](#Preparing-Text-Data-for-NLP) 
    
        - [Creating a matrix of features from text - Bag of N-gram Example](#Creating-a-matrix-of-features-from-text---Bag-of-N-gram-Example)
        
            - **_[Exercise 1 - convert corpus to matrix](#Exercise-1---convert-corpus-to-matrix)_**
        
        - [Calculating Word Counts](#Calculating-Word-Counts)
            
            - **_[Exercise 2 - getting word counts](#Exercise-2---getting-word-counts)_**
    
        - [Creating Text Corpus - choosing text to analyze](#Creating-Text-Corpus---choosing-text-to-analyze)
        - [Text Cleaning and Normalization](#Text-Cleaning-and-Normalization)
        
            - [First Description, Before Cleaning](#First-Description,-Before-Cleaning)
            - [First Description, After Cleaning](#First-Description,-After-Cleaning)
            
        - [Tokenizing text - breaking it into pieces](#Tokenizing-text---breaking-it-into-pieces)
        - [Removing meaningless text - Stopwords](#Removing-meaningless-text---Stopwords)
    
            - **_[Exercise 3 - practicing slicing](#Exercise-3---practicing-slicing)_**

    - [Topic Modeling on Cleaned Data](#Topic-Modeling-on-Cleaned-Data)
    
        - [Stemming and Lemmatization - Distilling text data](#Stemming-and-Lemmatization---Distilling-text-data)
        - [N-grams - Adding context by creating N-grams](#N-grams---Adding-context-by-creating-N-grams)
        - [TF-IDF - Weighting terms based on frequency](#TF-IDF---Weighting-terms-based-on-frequency)
        - **_[Exercise 4 - Refining a topic model](#Exercise-4---Refining-a-topic-model)_**
        - **_[Exercise 5 - Interpreting a model's "topics"](#Exercise-5---Interpreting-a-model's-"topics")_**
    
- [Supervised Learning: Document Classification](#Supervised-Learning:-Document-Classification)

    - [Supervised Learning - Prepare the Data](#Supervised-Learning---Prepare-the-Data)
    - [Prepare Data for Document Classification](#Prepare-Data-for-Document-Classification)
    - [Model Training - Train Document Classification Model](#Model-Training---Train-Document-Classification-Model)
    - [Model Evaluation - Precision and Recall](#Model-Evaluation---Precision-and-Recall)
    - [Model Evaluation - Feature Importances](#Model-Evaluation---Feature-Importances)
    
        - **_[Exercise 6 - interpreting feature importances](#Exercise-6---interpreting-feature-importances)_**
    
    - [Model Evaluation - Cross-validation](#Model-Evaluation---Cross-validation)
    
        - **_[Exercise 7 - Try a 5-fold cross-validation](#Exercise-7---Try-a-5-fold-cross-validation)_**
    
    - [Model Output - Examples of Document Classification](#Model-Output---Examples-of-Document-Classification)
    
- [Further Resources](#Further-Resources)
- Back to [Table of Contents](#Table-of-Contents)

## Introduction

- Back to [Table of Contents](#Table-of-Contents)

**Text analysis** is used to extract useful information from or summarize a large amount of unstructured text stored in documents. This opens up the opportunity of using text data alongside more conventional data sources (e.g. surveys and administrative data). The goal of text analysis is to take a large corpus of complex and unstructured text data and extract important and meaningful messages in a comprehensible way. 

Text analysis can help with the following tasks:

* **Information Retrieval**: Find relevant information in a large database, such as a systematic literature review, that would be very time-consuming for humans to do manually. 

* **Clustering and Text Categorization**: Summarize a large corpus of text by finding the most important phrases, using methods like topic modeling. 

* **Text Summarization**: Create category-sensitive text summaries of a large corpus of text. 

* **Machine Translation**: Translate documents from one language to another. 

In this tutorial, we are going to analyze patent descriptions using topic modeling. We will examine he content of the different patents and develop a way of clustering patents by key words.

### Learning Objectives

In this tutorial, you will...
* Learn how to transform a corpus of text into a structured matrix format so that we can apply natural language processing (NLP) methods
* Learn the basics and applications of topic modeling
* Learn how to do document tagging and evaluate the results

### Methods

 
### Glossary of Terms

Glossary of Terms:

* **Corpus**: A corpus is the set of all text documents used in your analysis; for example, your corpus of text may include hundreds of research articles.

* **Tokenize**: Tokenization is the process by which text is separated into meaningful terms or phrases. In English this is easy to do for individual words, as they are separated by whitespace; however, it can get more complicated to  automate determining which groups of words constitute meaningful phrases. 

* **Stemming**: Stemming is normalizing text by reducing all forms or conjugations of a word to the word's most basic form. In English, this can mean making a rule of removing the suffixes "ed" or "ing" from the end of all words, but it gets more complex. For example, "to go" is irregular, so you need to tell the algorithm that "went" and "goes" stem from a common lemma, and should be considered alternate forms of the word "go."

* **TF-IDF**: TF-IDF (term frequency-inverse document frequency) is an example of feature engineering where the most important words are extracted by taking account their frequency in documents and the entire corpus of documents as a whole.

* **Topic Modeling**: Topic modeling is an unsupervised learning method where groups of words that often appear together are clustered into topics. Typically, the words in one topic should be related and make sense (e.g. boat, ship, captain). Individual documents can fall under one topic or multiple topics. 

* **LDA**: LDA (Latent Dirichlet Allocation) is a type of probabilistic model commonly used for topic modeling. 

* **Stop Words**: Stop words are words that have little semantic meaning but occur very frequently, like prepositions, articles and common nouns. For example, every document (in English) will probably contain the words "and" and "the" many times. You will often remove them as part of preprocessing using a list of stop words.


## Setup

- Back to [Table of Contents](#Table-of-Contents)

In [1]:
%pylab inline 
import nltk
import ujson
import re
import time
import progressbar

import pandas as pd
from __future__ import print_function
from six.moves import zip, range 
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, roc_auc_score, auc
from sklearn import preprocessing
from collections import Counter, OrderedDict
from nltk.corpus import stopwords
from nltk import SnowballStemmer

Populating the interactive namespace from numpy and matplotlib


In [2]:
nltk.download('stopwords') #download the latest stopwords

[nltk_data] Downloading package stopwords to /Users/nj995/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Data Orientation

- Back to [Table of Contents](#Table-of-Contents)

Our dataset for this tutorial will be the patent description field that we downloaded during the class noteboook on APIs. The subset of patents we are considering are those assigned to companies that have offices in Kansas City, MO.

### Load the Data

- Back to [Table of Contents](#Table-of-Contents)

To start, we'll load the data into a pandas DataFrame from a CSV file.

In [3]:
df_patent_desc = pd.read_csv('../../output/patent_desc.csv')

/Users/nj995/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df_patent_desc['year'] = pd.to_datetime(df_patent_desc['patent_date'], errors = 'coerce').dt.year
df_patent_desc = df_patent_desc[df_patent_desc['year']>2010].reset_index(drop = True)
del df_patent_desc['year']

### Explore the Data

- Back to [Table of Contents](#Table-of-Contents)

Our Patent Description data table has 4 fields:

- `business_name` - business name that we cleaned/standardized in the Record Linkage notebook.
- `patent_number` - unique ID for each patent.
- `patent_title` - title of the patent.
- `patent_abstract` - A brief description of the patent content.

Let's take a look at examples of the values:

In [5]:
df_patent_desc.head()

,business_name,patent_number,patent_date,patent_title,patent_abstract
0,MICROSOFT CORP,7958069,2011-01-16,Time modulated generative probabilistic models...,Dependencies between different channels or dif...
1,MICROSOFT CORP,7978185,2011-02-18,Creating virtual replicas of physical objects,"The present invention extends to methods, syst..."
2,DIEBOLD INCORPORATED,7984847,2011-01-27,Banking system controlled by data bearing records,An automated banking machine operates responsi...
3,EASTMAN KODAK CO,7985684,2011-01-07,Actuating transistor including reduced channel...,A method of actuating a semiconductor device i...
4,TOYOTA MOTOR SALES USA INC,7986126,2011-02-18,Automated system for determining whether vehic...,A system may automatically identify whether a ...


In [6]:
print(df_patent_desc['patent_abstract'][0])

Dependencies between different channels or different services in a client or server may be determined from the observation of the times of the incoming and outgoing of the packets constituting those channels or services. A probabilistic model may be used to formally characterize these dependencies. The probabilistic model may be used to list the dependencies between input packets and output packets of various channels or services, and may be used to establish the expected strength of the causal relationship between the different events surrounding those channels or services. Parameters of the probabilistic model may be either based on prior knowledge, or may be fit using statistical techniques based on observations about the times of the events of interest. Expected times of occurrence between events may be observed, and dependencies may be determined in accordance with the probabilistic model.


In [7]:
df_patent_desc.describe()

,business_name,patent_number,patent_date,patent_title,patent_abstract
count,32139,32139,32139,32139,27498
unique,320,32136,1650,25469,25094
top,GENERAL ELECTRIC CO,9177480,2013-03-15,Display screen with graphical user interface,An insulated icemaking compartment is provided...
freq,6050,2,324,752,15


Looking at the `count` line, we note that some patents have no patent abstract. Let's restrict to patents where `patent_abstract` is not `NaN`.

In [8]:
df_patent_desc = df_patent_desc[df_patent_desc['patent_abstract'].isnull() == False]

## Topic Modeling

- Back to [Table of Contents](#Table-of-Contents)

We are going to apply topic modeling, an unsupervised learning method, to our corpus to find the high-level topics in our corpus as a "first go" for exploring our data. Through this process, we'll discuss how to clean and preprocess our data to get the best results.

Topic modeling is a broad subfield of machine learning and natural language processing. We are going to focus on a common modeling approach called Latent Dirichlet Allocation (LDA). 

To use topic modeling, we first have to assume that topics exist in our corpus, and that some small number of these topics can "explain" the corpus. Topics in this context refer to words from the corpus, in a list that is ranked by probability. A single document can be explained by multiple topics. For instance, a patent on a new medical robot would fall under the topic "robotics" as well as the topic "medicine". The set of topics used by a document is known as the document's allocation, hence, the name Latent Dirchlet Allocation, each document has an allocation of latent topics allocated by Dirchlet distribution. 

## Preparing Text Data for NLP

- Back to [Table of Contents](#Table-of-Contents)

The first important step in working with text data is cleaning and processing the data, which includes (but is not limited to):

- forming a corpus of text
- tokenization
- removing stop-words
- finding words co-located together (N-grams)
- stemming and lemmatization

Each of these steps will be discussed below. 

The ultimate goal is to transform our text data into a form an algorithm can work with, because a document or a corpus of text cannot be fed directly into an algorithm. Algorithms expect numerical feature vectors with certain fixed sizes, and can't handle documents, which are basically sequences of symbols with variable length. We will be transforming our text corpus into a *bag of n-grams* to be further analyzed. In this form our text data is represented as a matrix where each row refers to a specific job description (document) and each column is the occurence of a word (feature).

### Creating a matrix of features from text - Bag of N-gram Example

- Back to [Table of Contents](#Table-of-Contents)

Ultimately, we want to take our collection of documents, corpus, and convert it into a matrix. Fortunately, `sklearn` has a pre-built object, `CountVectorizer`, that can tokenize, eliminate stopwords, identify n-grams, and stem our corpus, and output a matrix in one step. Before we apply the vectorizer to our corpus of data, let's apply it to a toy example so that we see what the output looks like and how a bag of words is represented. 

In [9]:
def create_bag_of_words(corpus, 
                        NGRAM_RANGE = (0, 1), 
                        stop_words = None, 
                        stem = False, 
                        MIN_DF = 0.05, 
                        MAX_DF = 0.95, 
                        USE_IDF = False):

    """
    Turn a corpus of text into a bag-of-words.
    
    Parameters
    -----------
    corpus: ls
        test of documents in corpus    
    NGRAM_RANGE: tuple
        range of N-gram. Default (0,1)
    stop_words: ls
        list of commonly occuring words that have little semantic
        value
    stem: bool
        use a stemmer to stem words
    MIN_DF: float
       exclude words that have a frequency less than the threshold
    MAX_DF: float
        exclude words that have a frequency greater than the threshold
    USE_IDF: bool
        Re-weigh words according to the Term Frequency-Inverse Document Frequency 
        (emphasize words unique to a document, suppress words common throughout the corpus)
    
    Returns
    -------
    bag_of_words: scipy sparse matrix
        scipy sparse matrix of text
    features:
        list of words
    """
    #parameters for vectorizer 
    ANALYZER = "word" #unit of features are single words rather then phrases of words 
    STRIP_ACCENTS = 'unicode'
    
    if stem:
        stemmer = nltk.SnowballStemmer("english")
        tokenize = lambda x: [stemmer.stem(i) for i in x.split()]
    else:
        tokenize = None
    vectorizer = CountVectorizer(analyzer=ANALYZER,
                                 tokenizer=tokenize, 
                                 ngram_range=NGRAM_RANGE,
                                 stop_words = stop_words,
                                 strip_accents=STRIP_ACCENTS,
                                 min_df = MIN_DF,
                                 max_df = MAX_DF)
    
    bag_of_words = vectorizer.fit_transform( corpus ) #transform our corpus is a bag of words 
    features = vectorizer.get_feature_names()

    if USE_IDF:
        NORM = None #turn on normalization flag
        SMOOTH_IDF = True #prvents division by zero errors
        SUBLINEAR_IDF = True #replace TF with 1 + log(TF)
        transformer = TfidfTransformer(norm = NORM,smooth_idf = SMOOTH_IDF,sublinear_tf = True)
        #get the bag-of-words from the vectorizer and
        #then use TFIDF to limit the tokens found throughout the text 
        tfidf = transformer.fit_transform(bag_of_words)
        
        return tfidf, features
    else:
        return bag_of_words, features

In [10]:
# create example corpus.
toy_corpus = ['this is document one', 'this is document two', 'text analysis on documents is fun']

In [11]:
# convert to bag of words
toy_bag_of_words, toy_features = create_bag_of_words( toy_corpus )

In [12]:
# review - our corpus:
toy_corpus

['this is document one',
 'this is document two',
 'text analysis on documents is fun']

In [13]:
# features derived from the corpus
toy_features

['analysis',
 'document',
 'documents',
 'fun',
 'on',
 'one',
 'text',
 'this',
 'two']

In [14]:
# bag of words that results:
np_bag_of_words = toy_bag_of_words.toarray()
np_bag_of_words

array([[0, 1, 0, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 1],
       [1, 0, 1, 1, 1, 0, 1, 0, 0]], dtype=int64)

Our data has been transformed from a document into a 3 x 9 matrix, where each row in the matrix corresponds to a document, and each column corresponds to a feature (in the order they appear in `toy_features`). A 1 indicates the existence of the feature or word in the document, and a 0 indicates the word is not present.

It is very common that this representation will be a "sparse" matrix, or a matrix that has a lot of 0s. With sparse matrices, it is often more efficient to keep track of which values *aren't* 0 and where those non-zero entries are located, rather than to save the entire matrix. To save space, the `scipy` library has special ways of storing sparse matrices in an efficient way. 

Our toy corpus is now ready to be analyzed. We used this toy example to illustrate how a document is turned into a matrix to be used in text analysis. When you're applying this to real text data, the matrix will be much larger and harder to interpret, but it's important that you know the process. 

__Exercise 1 - convert corpus to matrix__

- Back to [Table of Contents](#Table-of-Contents)

To check your knowledge, make your own toy corpus and turn it into a matrix.

In [15]:
#solution
exercise_corpus = ['Batman is friends with Superman', 
                   'Superman is enemies with Lex Luthor',
                   'Batman is enemies with Lex Luthor' ] 
exercise_bag_of_words, exercise_features = create_bag_of_words(exercise_corpus)

In [16]:
# convert bag of words to array
np_bag_of_words = exercise_bag_of_words.toarray()

In [17]:
# show features:
exercise_features

['batman', 'enemies', 'friends', 'lex', 'luthor', 'superman']

In [18]:
# output derived bag of words:
np_bag_of_words

array([[1, 0, 1, 0, 0, 1],
       [0, 1, 0, 1, 1, 1],
       [1, 1, 0, 1, 1, 0]], dtype=int64)

### Calculating Word Counts

- Back to [Table of Contents](#Table-of-Contents)

As an initial look into the data, we can examine the most frequently occuring words in our corpus. We can sum the columns of the bag_of_words and then convert to a numpy array. From here we can zip the features and word_count into a dictionary, and display the results.

In [19]:
def get_word_counts(bag_of_words, feature_names):

    """
    Get the ordered word counts from a bag_of_words
    
    Parameters
    ----------
    bag_of_words: obj
        scipy sparse matrix from CounterVectorizer
    feature_names: ls
        list of words
        
    Returns
    -------
    word_counts: dict
        Dictionary of word counts
    """

    # convert bag of words to array
    np_bag_of_words = bag_of_words.toarray()
    
    # calculate word count.
    word_count = np.sum(np_bag_of_words,axis=0)
    
    # convert to flattened array.
    np_word_count = np.asarray(word_count).ravel()
    
    # create dict of words mapped to count of occurrences of each word.
    dict_word_counts = dict( zip(feature_names, np_word_count) )
    
    # Create ordered dictionary
    orddict_word_counts = OrderedDict( sorted(dict_word_counts.items(), key=lambda x: x[1], reverse=True), )
    
    return orddict_word_counts

In [20]:
# get ordered word counts for our example corpus.
get_word_counts(toy_bag_of_words, toy_features)

OrderedDict([('document', 2),
             ('this', 2),
             ('analysis', 1),
             ('documents', 1),
             ('fun', 1),
             ('on', 1),
             ('one', 1),
             ('text', 1),
             ('two', 1)])

Note that the words "document" and "documents" both appear separately in the list. Should they be treated as the same words, since one is just the plural of the other, or should they be considered distinct words? These are the types of decisions you will have to make in your preprocessing steps.

__Exercise 2 - getting word counts__

- Back to [Table of Contents](#Table-of-Contents)

Get the word counts of your exercise corpus.


In [21]:
get_word_counts(exercise_bag_of_words, exercise_features)

OrderedDict([('batman', 2),
             ('enemies', 2),
             ('lex', 2),
             ('luthor', 2),
             ('superman', 2),
             ('friends', 1)])

## Creating Text Corpus - choosing text to analyze

- Back to [Table of Contents](#Table-of-Contents)

First we need to form our corpus, or the set of all patent abstracts. We can pull out the array of descriptions from the data frame using the data frame's `.values` attribute. 

In [22]:
corpus = df_patent_desc['patent_abstract'].values #pull all the descriptions and put them in a numpy array 
# corpus = corpus[:1000]

In [23]:
len(corpus)

27498

## Model topics with LDA

- Back to [Table of Contents](#Table-of-Contents)

As a start to modeling topics, we define below a function "create_topics" that uses Latent Dirichlet Allocation (LDA) to find topics.

In [24]:
def create_topics(tfidf, features, N_TOPICS=3, N_TOP_WORDS=5):
    """
    Given a matrix of features of text data generate topics
    
    Parameters
    -----------
    tfidf: scipy sparse matrix
        sparse matrix of text features
    N_TOPICS: int
        number of topics (default 10)
    N_TOP_WORDS: int
        number of top words to display in each topic (default 10)
        
    Returns
    -------
    ls_keywords: ls
        list of keywords for each topics
    doctopic: array
        numpy array with percentages of topic that fit each category
    N_TOPICS: int
        number of assumed topics
    N_TOP_WORDS: int
        Number of top words in a given topic. 
    """
    
    i=0
    lda = LatentDirichletAllocation(n_components= N_TOPICS,
                                    learning_method='online') #create an object that will create 5 topics
    i+=1
    doctopic = lda.fit_transform( tfidf )
    i+=1

    ls_keywords = []
    for i,topic in enumerate(lda.components_):
        word_idx = np.argsort(topic)[::-1][:N_TOP_WORDS]
        keywords = ', '.join( features[i] for i in word_idx)
        ls_keywords.append(keywords)
        print(i, keywords)
        i+=1
            
    return ls_keywords, doctopic

Create a bag of words and set of features from our social services corpus:

In [25]:
corpus_bag_of_words, corpus_features = create_bag_of_words(corpus)

Let's examine our features. 

In [26]:
corpus_features

['access',
 'along',
 'also',
 'an',
 'and',
 'apparatus',
 'application',
 'are',
 'as',
 'assembly',
 'associated',
 'at',
 'based',
 'be',
 'being',
 'between',
 'by',
 'can',
 'communication',
 'comprises',
 'comprising',
 'computer',
 'configured',
 'connected',
 'control',
 'corresponding',
 'coupled',
 'data',
 'described',
 'determine',
 'determined',
 'determining',
 'device',
 'devices',
 'different',
 'disclosed',
 'disposed',
 'during',
 'each',
 'embodiment',
 'embodiments',
 'end',
 'example',
 'first',
 'flow',
 'for',
 'form',
 'from',
 'further',
 'has',
 'having',
 'herein',
 'if',
 'image',
 'in',
 'include',
 'includes',
 'including',
 'information',
 'input',
 'interface',
 'into',
 'invention',
 'is',
 'least',
 'material',
 'may',
 'method',
 'methods',
 'more',
 'multiple',
 'network',
 'non',
 'not',
 'of',
 'on',
 'one',
 'or',
 'other',
 'output',
 'over',
 'plurality',
 'portion',
 'position',
 'power',
 'present',
 'process',
 'processing',
 'provide',
 'pr

The first aspect to notice about the feature list is that the first few entries are numbers that have no real semantic meaning. The feature lists also includes numerous other useless words, such as prepositions and articles, that will just add noise to our analysis. 

We can also notice the words *comprises* and *comprising*, or the words *form* and *formed*, are close enough to each other that it might not make sense to treat them as entirely separate words. Part of your cleaning and preprocessing duties will be manually inspecting your lists of features, seeing where these issues arise, and making decisions to either remove them from your analysis or address them separately. 

Let's get the count of the number of times that each of the words appears in our corpus.

In [27]:
get_word_counts(corpus_bag_of_words, corpus_features)

OrderedDict([('of', 105151),
             ('to', 86662),
             ('and', 85413),
             ('is', 42789),
             ('in', 41213),
             ('an', 39636),
             ('for', 34945),
             ('first', 27154),
             ('one', 24675),
             ('or', 23866),
             ('system', 22250),
             ('at', 21835),
             ('with', 21212),
             ('includes', 20639),
             ('that', 20534),
             ('second', 20168),
             ('from', 19331),
             ('on', 18260),
             ('are', 16936),
             ('be', 16488),
             ('device', 16247),
             ('data', 15537),
             ('may', 15085),
             ('least', 14677),
             ('method', 13304),
             ('by', 13262),
             ('plurality', 10976),
             ('more', 10122),
             ('configured', 9744),
             ('based', 9640),
             ('can', 9084),
             ('each', 8897),
             ('as', 8498),
             ('i

Our top words are articles, prepositions and conjunctions that are not informative whatsoever, so we're probably not going to come up with anything interesting ("garbage in, garbage out"). 

Nevertheless, let's forge blindly ahead and try to create topics, and see the quality of the results that we get.

In [28]:
ls_corpus_keywords, corpus_doctopic = create_topics(corpus_bag_of_words, corpus_features)

0 of, and, one, to, at
1 and, to, first, of, second
2 to, and, of, in, be


These topics don't give us any real insight to what the data contains - one of the topics is "and, the, to, of, in"! Accroding to the patents in your dataset, some might hint to subjects ("article", "wall", "vending", etc.) but the signal is being swamped by the noise. 

We'll have to clean and process our data to get any meaningful information out of this text. 

### Text Cleaning and Normalization

- Back to [Table of Contents](#Table-of-Contents)

To clean and normalize text, we'll remove all special characters, numbers, and punctuation, so we're left with only the words themselves. Then we will make all the text lowercase; this uniformity will ensure that the algorithm doesn't treat "the" and "The" as different words, for example. 

To remove the special characters, numbers and punctuation we will use regular expressions. 


**Regular Expressions**, or "regexes" for short, let you find all the words or phrases in a document or text file that match a certain pattern. These rules are useful for pulling out useful information from a large amount of text. For example, if you want to find all email addresses in a document, you might look for everything that looks like *some combination of letters, _, .* followed by *@*, followed by more letters, and ending in *.com* or *.edu*. If you want to find all the credit card numbers in a document, you might look for everywhere you see the pattern "four numbers, space, four numbers, space, four numbers, space, four numbers." Regexes are also helpful if you are scraping information from websites, because you can use them to separate the content from the HTML code used for formatting the website.

A full tutorial on regular expressions would be outside the scope of this tutorial, but many good tutorials that can be found on-line. [regex101.com](regex101.com) is also a great interactive tool for developing and checking regular expressions.

>"Some people, when confronted with a problem, think 
>'I know, I'll use regular expressions.'   Now they have two problems."
> -- Jaime Zawinski

*A word of warning:* Regexes can work much more quickly than plain text sorting; however, if your regular expressions are becoming overly complicated, it's a good idea to find a simpler way to do what you want to do. Any developer should keep in mind there is a trade-off between optimization and understandability. The general philosophy of programming in Python is that your code is meant to be as understandable by *people* as much as possible, because human time is more valuable than computer time. You should therefore lean toward understandability rather than overly optimizing your code to make it run as quickly as possible. Your future-self, code-reviewers, people who inherit your code, and anyone else who has to make sense of your code in the future will appreciate it. 

For our purposes, we are going to use a regular expression to match all characters that are not letters -- punctuation, quotes, special characters and numbers -- and replace them with spaces. Then we'll make all of the remaining characters lowercase.  

We will be using the `re` library in python for regular expression matching.

In [29]:
#get rid of the punctuations and set all characters to lowercase
RE_PREPROCESS = re.compile( r'\W+|\d+' ) #the regular expressions that matches all non-characters

#get rid of punctuation and make everything lowercase
#the code below works by looping through the array of text ("corpus")
#for a given piece of text ( "description" ) we invoke the `re.sub` command 
#the `re.sub` command takes 3 arguments: (1) the regular expression to match, 
#(2) what we want to substitute in place of that matching string (' ', a space)
#and (3) the text we want to apply this to. 
#we then invoke the `lower()` method on the output of the `re.sub` command
#to make all the remaining characters lowercase.
#the result is a list, where each entry in the list is a cleaned version of the
#corresponding entry in the original corpus.
#we then make the list into a numpy array to use it in analysis

processed_corpus = np.array( [ re.sub( RE_PREPROCESS, ' ', description ).lower() for description in corpus ] )

Next, let's look at an example of the results of this cleanup.

__First Description, Before Cleaning__

- Back to [Table of Contents](#Table-of-Contents)

First, we'll look at the first description in our corpus, before it was cleaned:

In [30]:
corpus[0]

'Dependencies between different channels or different services in a client or server may be determined from the observation of the times of the incoming and outgoing of the packets constituting those channels or services. A probabilistic model may be used to formally characterize these dependencies. The probabilistic model may be used to list the dependencies between input packets and output packets of various channels or services, and may be used to establish the expected strength of the causal relationship between the different events surrounding those channels or services. Parameters of the probabilistic model may be either based on prior knowledge, or may be fit using statistical techniques based on observations about the times of the events of interest. Expected times of occurrence between events may be observed, and dependencies may be determined in accordance with the probabilistic model.'

This text includes a lot of useful information, but also includes some things we don't want or need. There are some weird special characters (...). There are also some numbers, which are informative and interesting to a human reading the text (phone numbers, addresses, ...), but when we break down the documents into individual words, the numbers will become meaningless. We'll also want to remove all punctuation, so that we can say any two things separated by a space are individual words.

__First Description, After Cleaning__

- Back to [Table of Contents](#Table-of-Contents)

Now, let's look at this text after cleaning:

In [31]:
processed_corpus[0]

'dependencies between different channels or different services in a client or server may be determined from the observation of the times of the incoming and outgoing of the packets constituting those channels or services a probabilistic model may be used to formally characterize these dependencies the probabilistic model may be used to list the dependencies between input packets and output packets of various channels or services and may be used to establish the expected strength of the causal relationship between the different events surrounding those channels or services parameters of the probabilistic model may be either based on prior knowledge or may be fit using statistical techniques based on observations about the times of the events of interest expected times of occurrence between events may be observed and dependencies may be determined in accordance with the probabilistic model '

All lowercase, all numbers and special characters have been removed. Out text is now normalized.

### Tokenizing text - breaking it into pieces

- Back to [Table of Contents](#Table-of-Contents)

Now that we've cleaned our text, we can *tokenize* it by deciding which words or phrases are the most meaningful. Normally the `CountVectorizer` handles this for us, but in this case, we'll split our text into individual words manually to show how it is done.

To go from a whole document to a list of individual words, we can use the `.split()` command. By default, this command splits based on spaces in between words, so we don't need to specify that explicitly.  

In [32]:
tokens = processed_corpus[0].split()

In [33]:
tokens

['dependencies',
 'between',
 'different',
 'channels',
 'or',
 'different',
 'services',
 'in',
 'a',
 'client',
 'or',
 'server',
 'may',
 'be',
 'determined',
 'from',
 'the',
 'observation',
 'of',
 'the',
 'times',
 'of',
 'the',
 'incoming',
 'and',
 'outgoing',
 'of',
 'the',
 'packets',
 'constituting',
 'those',
 'channels',
 'or',
 'services',
 'a',
 'probabilistic',
 'model',
 'may',
 'be',
 'used',
 'to',
 'formally',
 'characterize',
 'these',
 'dependencies',
 'the',
 'probabilistic',
 'model',
 'may',
 'be',
 'used',
 'to',
 'list',
 'the',
 'dependencies',
 'between',
 'input',
 'packets',
 'and',
 'output',
 'packets',
 'of',
 'various',
 'channels',
 'or',
 'services',
 'and',
 'may',
 'be',
 'used',
 'to',
 'establish',
 'the',
 'expected',
 'strength',
 'of',
 'the',
 'causal',
 'relationship',
 'between',
 'the',
 'different',
 'events',
 'surrounding',
 'those',
 'channels',
 'or',
 'services',
 'parameters',
 'of',
 'the',
 'probabilistic',
 'model',
 'may',
 'be

### Removing meaningless text - Stopwords

- Back to [Table of Contents](#Table-of-Contents)

Stopwords are words that are found commonly throughout a text and carry little semantic meaning. Examples of common stopwords are prepositions ("to", "on", "in"), articles ("the", "an", "a"), conjunctions ("and", "or", "but") and common nouns. For example, the words *the* and *of* are totally ubiquitous, so they won't serve as meaningful features, whether to distinguish documents from each other or to tell what a given document is about. You may also run into words that you want to remove based on where you obtained your corpus of text or what it's about. There are many lists of common stopwords available for you to use, both for general documents and for specific contexts, so you don't have to start from scratch.   

We can eliminate stopwords by checking all the words in our corpus against a list of commonly occuring stopwords that comes with NLTK.

In [34]:
eng_stopwords = stopwords.words('english')

In [35]:
eng_stopwords

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [36]:
#sample of stopwords
#this is an example of slicing where we implicitly start at the beginning and move to the end
#we select every 10th entry in the array
eng_stopwords[::10]

['i',
 "you've",
 'himself',
 'they',
 'that',
 'been',
 'a',
 'while',
 'through',
 'in',
 'here',
 'few',
 'own',
 'just',
 're',
 'doesn',
 'ma',
 "shouldn't"]

Notice that this list includes "weren" and "hasn" as well as single letters ("t"). Why do you think these are contained in the list of stopwords?

__Exercise 3 - practicing slicing__

- Back to [Table of Contents](#Table-of-Contents)

Try slicing to retrieve every 5th word.

In [37]:
eng_stopwords[::5]

['i',
 'our',
 "you've",
 'yourself',
 'himself',
 'herself',
 'they',
 'what',
 'that',
 'is',
 'been',
 'having',
 'a',
 'if',
 'while',
 'with',
 'through',
 'below',
 'in',
 'under',
 'here',
 'how',
 'few',
 'such',
 'own',
 'very',
 'just',
 'now',
 're',
 "aren't",
 'doesn',
 "hasn't",
 'ma',
 'needn',
 "shouldn't",
 'won']

##  Topic Modeling on Cleaned Data

- Back to [Table of Contents](#Table-of-Contents)

Now that we've cleaned up our data a little bit, let's see what our bag of words looks like.

In [38]:
# create bag of words from processed_corpus
processed_bag_of_words, processed_features = create_bag_of_words( processed_corpus, stop_words = eng_stopwords )
dict_processed_word_counts = get_word_counts( processed_bag_of_words, processed_features )
dict_processed_word_counts

OrderedDict([('first', 27154),
             ('one', 24675),
             ('system', 22250),
             ('includes', 20639),
             ('second', 20168),
             ('device', 16255),
             ('data', 15537),
             ('may', 15085),
             ('least', 14677),
             ('method', 13304),
             ('plurality', 10976),
             ('configured', 9744),
             ('based', 9640),
             ('image', 7968),
             ('user', 7614),
             ('provided', 7023),
             ('portion', 6688),
             ('surface', 6178),
             ('information', 5856),
             ('associated', 5742),
             ('include', 5692),
             ('methods', 5660),
             ('network', 5272),
             ('including', 5187),
             ('also', 5185),
             ('communication', 5140),
             ('signal', 5095),
             ('control', 5081),
             ('power', 5030),
             ('using', 4970),
             ('within', 4915),
          

Much better! Now this is starting to look like a reasonable representation of our corpus of text. 

We mentioned that, in addition to stopwords that are common across all types of text analysis problems, there wil also be specific stopwords based on the context of your domain. Notice how the top words include words like "..."? It makes sense that these words are so common - so they won't be very helpful in analysis. 

One quick way to remove some of these domain-specific stopwords is by dropping some of your most frequent words. We'll start out by dropping the top 20. You'll want to change this number, playing with making it bigger and smaller, to see how it affects your resulting topics.

In [39]:
# get top 20 stopwords (slice from start through 20 items in list)
top_20_words = list(dict_processed_word_counts.keys())[:20]

# create new stopword list by combining default stopwords with our top 20.
domain_specific_stopwords = eng_stopwords + top_20_words

# make a new bag of words excluding custom stopwords.
processed_bag_of_words, processed_features = create_bag_of_words(processed_corpus,
                                                                 stop_words=domain_specific_stopwords)


In [40]:
# what do we have now?
dict_processed_word_counts = get_word_counts(processed_bag_of_words, processed_features)
dict_processed_word_counts

OrderedDict([('include', 5692),
             ('methods', 5660),
             ('network', 5272),
             ('including', 5187),
             ('also', 5185),
             ('communication', 5140),
             ('signal', 5095),
             ('control', 5081),
             ('power', 5030),
             ('using', 4970),
             ('within', 4915),
             ('assembly', 4697),
             ('systems', 4559),
             ('storage', 4500),
             ('coupled', 4443),
             ('set', 4392),
             ('end', 4384),
             ('disclosed', 4321),
             ('flow', 4224),
             ('wherein', 4066),
             ('time', 4032),
             ('sensor', 3890),
             ('devices', 3787),
             ('position', 3656),
             ('embodiments', 3617),
             ('apparatus', 3546),
             ('access', 3425),
             ('computer', 3414),
             ('provide', 3409),
             ('material', 3375),
             ('used', 3339),
             ('a

This is a bit better - although we still see some words that are probably very common ("..."), words like "..." will probably help us come up with more specific categories within the broader realm of social services. Let's see what topics we produce.

In [41]:
processed_keywords, processed_doctopic = create_topics(processed_bag_of_words, 
                                                       processed_features)

0 control, signal, power, sensor, position
1 assembly, storage, coupled, end, flow
2 methods, network, systems, communication, using


Now we are starting to get somewhere! We can manipulate the number of topics we want to find and the number of words to use for each topic to see if we can understand more from our corpus. 

In [42]:
# look for 5 topics, include 10 words in each.
processed_keywords, processed_doctopic = create_topics(processed_bag_of_words, 
                                                       processed_features, 
                                                       N_TOPICS = 5,
                                                       N_TOP_WORDS= 10)

0 methods, systems, set, sensor, time, using, include, embodiments, disclosed, used
1 communication, control, network, power, access, application, interface, source, via, computer
2 coupled, devices, output, input, response, disposed, receive, determine, multiple, also
3 assembly, storage, end, flow, position, within, along, connected, including, apparatus
4 signal, wherein, material, comprises, comprising, receiving, non, form, including, received


Some structure is starting to reveal itself - .... Adding more topics has revealed to larger subtopics. Let's see if increasing the number of topics gives us more information.

However, we can see that .... are still present -  This is an iterative process - after seeing the results of some analysis, you will need to go back to the preprocessing step and add more words to your list of stopwords or change how you cleaned the data.

Now let's try 10 topics with 15 words each:

In [43]:
# 10 topics, 15 words each
processed_keywords, processed_doctopic = create_topics(processed_bag_of_words, 
                                                       processed_features, 
                                                       N_TOPICS = 10,
                                                       N_TOP_WORDS= 15)

0 coupled, output, input, receive, determined, response, determine, also, received, provide, including, via, embodiment, described, disclosed
1 sensor, position, within, apparatus, source, disposed, along, also, comprises, via, use, wherein, including, providing, corresponding
2 assembly, wherein, comprises, comprising, two, including, form, also, disclosed, within, different, embodiment, using, coupled, providing
3 communication, storage, control, access, selected, via, received, within, corresponding, providing, determine, provide, also, including, embodiment
4 time, computer, receiving, determining, using, multiple, received, including, used, determine, also, processing, providing, via, use
5 end, set, include, connected, also, including, corresponding, disclosed, disposed, provide, received, wherein, using, embodiment, used
6 network, application, interface, process, processing, via, using, response, within, also, used, described, computer, receiving, provide
7 methods, systems, di

This looks like a good amount of topics for now. Some of the top words are quite similar, like <span style="background-color: #FFFF00"> "...." </span> Let's move to stemming and lemmatization.

### Stemming and Lemmatization - Distilling text data

- Back to [Table of Contents](#Table-of-Contents)

We can further process our text through *stemming and lemmatization*, or replacing words with their root or simplest form. For example "systems," "systematic," and "system" are all different words, but we can replace all these words with "system" without sacrificing much meaning. 

- A **lemma** is the original dictionary form of a word (e.g. the lemma for "lies," "lied," and "lying" is "lie").
- The process of turning a word into its simplest form is **stemming**. There are several well known stemming algorithms -- Porter, Snowball, Lancaster -- that all have their respective strengths and weaknesses.

For this tutorial, we'll use the Snowball Stemmer:

In [44]:
# Examples of how a Stemmer works:
stemmer = SnowballStemmer("english")
print(stemmer.stem('lies'))
print(stemmer.stem("lying"))
print(stemmer.stem('systematic'))
print(stemmer.stem("running"))

lie
lie
systemat
run


Let's try creating a bag of stemmed words.

In [45]:
# include stemming when creating our bag of words.
processed_bag_of_words, processed_features = create_bag_of_words(processed_corpus,
                                                                 stop_words=domain_specific_stopwords,
                                                                 stem=True)

In [46]:
# create topics with stemmed words.
processed_keywords, processed_doctopic = create_topics(processed_bag_of_words, 
                                                       processed_features, 
                                                       N_TOPICS = 10,
                                                       N_TOP_WORDS= 15)

0 use, provid, embodi, applic, present, relat, particular, describ, disclos, invent, exampl, includ, multipl, contain, herein
1 compon, layer, end, includ, member, form, materi, structur, compris, support, connect, plural, provid, wherein, contain
2 process, sensor, time, detect, dure, oper, use, within, perform, provid, apparatus, includ, compris, indic, base
3 includ, surfac, posit, assembl, flow, fluid, defin, extend, within, dispos, direct, plural, provid, along, form
4 configur, includ, power, coupl, electr, sourc, oper, generat, receiv, provid, also, direct, output, wherein, compris
5 control, signal, set, valu, modul, output, input, includ, receiv, generat, base, oper, determin, provid, respect
6 imag, print, use, includ, receiv, non, identifi, generat, form, determin, correspond, locat, appli, base, compris
7 devic, storag, element, interfac, oper, connect, store, communic, includ, provid, compris, via, use, embodi, apparatus
8 plural, base, determin, includ, display, comput, s

What do we think of these topics?

### N-grams - Adding context by creating N-grams

- Back to [Table of Contents](#Table-of-Contents)

Obviously, reducing a document to a bag of words means losing much of its meaning - we put words in certain orders, and group words together in phrases and sentences, precisely to give them more meaning. If you follow the processing steps we've gone through so far, splitting your document into individual words and then removing stopwords, you'll completely lose all phrases like "kick the bucket," "commander in chief," or "sleeps with the fishes." 

One way to address this is to break down each document similarly, but rather than treating each word as an individual unit, treat each group of 2 words, or 3 words, or *n* words, as a unit. We call this a "bag of *n*-grams," where *n* is the number of words in each chunk. Then you can analyze which groups of words commonly occur together (in a fixed order). 

Let's transform our corpus into a bag of n-grams with *n*=2: a bag of 2-grams, AKA a bag of bi-grams.

In [47]:
# create bag of words with stemmed words and 2-grams (NGRAM_RANGE = (0, 2)).
processed_bag_of_words, processed_features = create_bag_of_words(processed_corpus,
                                                                 stop_words=domain_specific_stopwords,
                                                                 stem=True,
                                                                 NGRAM_RANGE=(0,2))

# Create topics.
processed_keywords, processed_doctopic = create_topics(processed_bag_of_words, 
                                                       processed_features, 
                                                       N_TOPICS = 10,
                                                       N_TOP_WORDS= 15)

0 signal, oper, print, receiv, detect, output, input, includ, respons, generat, determin, transmit, base, also, use
1 plural, configur, flow, sensor, posit, includ, valu, modul, respect, provid, coupl, compris, direct, oper, receiv
2 includ, surfac, assembl, compon, end, fluid, extend, dispos, defin, configur, within, connect, coupl, form, also
3 base, determin, display, select, locat, element, includ, associ, plural, provid, differ, indic, exampl, relat, embodi
4 control, power, layer, electr, sourc, includ, configur, coupl, oper, generat, connect, provid, dure, wherein, direct
5 devic, communic, network, comput, access, receiv, interfac, request, includ, identifi, provid, use, configur, oper, associ
6 member, structur, support, non, along, includ, form, connect, provid, extend, configur, defin, coupl, appli, within
7 imag, set, identifi, generat, use, includ, comput, correspond, multipl, determin, within, receiv, defin, form, two
8 process, storag, time, inform, applic, use, store, p

We can see that this lets us uncover patterns that we couldn't when we just used a bag of words: "north shore" and "domest violenc" come up as words. Note that this still includes the individual words, as well as the bi-grams.

### TF-IDF - Weighting terms based on frequency

- Back to [Table of Contents](#Table-of-Contents)

A final step in cleaning and processing our text data is **Term Frequency-Inverse Document Frequency (TF-IDF)**. TF-IDF is based on the idea that the words (or terms) that are most related to a certain topic will occur frequently in documents on that topic, and infrequently in unrelated documents.  TF-IDF re-weights words so that we emphasize words that are unique to a document and suppress words that are common throughout the corpus by inversely weighting terms based on their frequency within the document and across the corpus.

Let's look at how using TF-IDF affects our bag of words:

In [48]:
# create bag of words including TF-IDF weighting.
processed_bag_of_words, processed_features = create_bag_of_words( processed_corpus,
                                                                  stop_words = domain_specific_stopwords,
                                                                  stem = True,
                                                                  NGRAM_RANGE = ( 0, 2 ),
                                                                  USE_IDF = True )

In [49]:
# let's see what we have:
dict_word_counts = get_word_counts( processed_bag_of_words, processed_features )
dict_word_counts

OrderedDict([('includ', 37362.806436766521),
             ('devic', 29349.613251516152),
             ('provid', 26467.771481305514),
             ('base', 23070.333663862442),
             ('use', 22968.451746546787),
             ('configur', 22789.679819976503),
             ('receiv', 21872.12126534888),
             ('plural', 21607.507199707241),
             ('determin', 20664.249517194799),
             ('imag', 20053.247726168189),
             ('control', 19790.954325824096),
             ('oper', 18476.836573603494),
             ('compris', 16664.660192233747),
             ('generat', 16271.87362308214),
             ('communic', 16032.436246516616),
             ('embodi', 15728.981937480541),
             ('associ', 15438.616421941037),
             ('posit', 15425.058735836918),
             ('surfac', 15110.502585307346),
             ('process', 14954.537044628198),
             ('comput', 14843.128752121338),
             ('signal', 14804.816418977385),
             

The words counts have been reweighted to emphasize the more meaningful words of the corpus, while de-emphasizing the words that are found commonly throughout the corpus.

How does this affect our topics?

In [50]:
processed_keywords, processed_doctopic = create_topics(processed_bag_of_words, 
                                                       processed_features, 
                                                       N_TOPICS = 5,
                                                       N_TOP_WORDS= 10)

0 communic, devic, network, locat, detect, signal, applic, element, modul, interfac
1 coupl, configur, includ, assembl, power, flow, control, sensor, fluid, electr
2 determin, base, associ, identifi, set, comput, display, time, devic, valu
3 inform, compon, layer, materi, request, non, compris, appli, form, structur
4 imag, surfac, end, extend, member, print, contain, support, includ, form


---
__Exercise 4 - Refining a topic model__

- Back to [Table of Contents](#Table-of-Contents)

You can only develop an intuition for the right number of topics and topic words suitable for a given problem by iterating until you find a good match. 

Change the number of topics and topic words until you get an intution of how many words and topics are enough.

In [51]:
exercise_keywords, exercise_doctopic = create_topics( processed_bag_of_words, 
                                                      processed_features, 
                                                      N_TOPICS = 5,
                                                      N_TOP_WORDS= 25 )

0 imag, use, present, embodi, process, print, disclos, provid, non, appli, invent, addit, relat, herein, particular, allow, contain, reduc, describ, compris, exampl, includ, apparatus, differ, direct
1 control, coupl, configur, assembl, includ, power, flow, fluid, electr, storag, connect, sourc, also includ, oper, also, devic, provid, posit, direct, plural, within, communic, via, receiv, generat
2 signal, determin, identifi, set, base, detect, sensor, time, valu, indic, input, correspond, locat, generat, receiv, output, respons, associ, includ, plural, perform, comput, devic, use, oper
3 devic, network, communic, display, inform, access, modul, request, interfac, receiv, comput, associ, transmit, applic, store, base, determin, provid, configur, control, select, includ, oper, plural, respons
4 surfac, end, form, layer, extend, posit, includ, member, element, defin, materi, structur, support, compon, dispos, along, compris, plural, within, wherein, contain, locat, provid, respect, direct

In [52]:
exercise_keywords, exercise_doctopic = create_topics( processed_bag_of_words, 
                                                      processed_features, 
                                                      N_TOPICS = 10,
                                                      N_TOP_WORDS= 25 )

0 set, also, relat, embodi, describ, also includ, particular, herein, disclos, present, base, use, includ, associ, provid, select, determin, respect, generat, devic, plural, inform, receiv, identifi, oper
1 compris, form, wherein, layer, materi, plural, invent, provid, present, includ, embodi, use, base, select, disclos, also, differ, contain, receiv, devic, dure, process, within, generat, extend
2 imag, time, valu, storag, identifi, determin, comput, correspond, perform, base, store, generat, use, plural, associ, devic, differ, receiv, includ, inform, process, select, oper, provid, dure
3 signal, control, display, modul, output, input, select, configur, plural, multipl, generat, differ, receiv, devic, oper, includ, coupl, base, provid, indic, determin, respons, associ, correspond, apparatus
4 compon, process, applic, interfac, accord, use, provid, dure, present, oper, embodi, perform, allow, includ, generat, direct, configur, base, associ, control, receiv, determin, request, identifi,

In [53]:
#grab the topic_id of the majority topic for each document and store it in a list
ls_topic_id = [np.argsort(processed_doctopic[comment_id])[::-1][0] for comment_id in range(len(corpus))]
df_patent_desc['topic_id'] = ls_topic_id #add to the dataframe so we can compare with the job titles

Now that each row is tagged with a topic ID. Let's see how well the topics explain the social services by looking at the first topic, and seeing how similar the social services within that topic are to each other.

In [54]:
topic_num = 0
print(processed_keywords[topic_num])
df_patent_desc[df_patent_desc.topic_id == topic_num].head(10)

communic, devic, network, locat, detect, signal, applic, element, modul, interfac


,business_name,patent_number,patent_date,patent_title,patent_abstract,topic_id
8,ORACLE AMERICA INC,8006016,2011-01-18,Hiding system latencies in a throughput networ...,A method for addressing system latency within ...,0
19,ORACLE AMERICA INC,8039308,2011-01-13,Integrated-circuit package for proximity commu...,Embodiments of a multi-chip module (MCM) are d...,0
30,MICROSOFT CORP,8065421,2011-02-15,Reciprocal public trust relationship,Publicly accessible linking information is to ...,0
57,EASTMAN KODAK CO,8098403,2011-05-11,Color error diffusion with error signal offset,A method for multi-toning an input digital ima...,0
58,MICROSOFT CORP,8099494,2011-03-31,Tuning and optimizing distributed systems with...,A system for automatically adjusting operation...,0
69,ORACLE AMERICA INC,8108461,2011-07-08,Method and system for processing a request sen...,A method for processing a request sent over a ...,0
72,MICROSOFT CORP,8112116,2011-05-11,Bidirectional dynamic offloading of tasks betw...,One or more functions are exposed by a mobile ...,0
77,MICROSOFT CORP,8112556,2011-05-31,Method and apparatus for detecting the type of...,"A peripheral device, with first and second com...",0
81,MICROSOFT CORP,8116234,2011-01-31,Detection of home network configuration problems,A diagnostic tool for identifying a configurat...,0
93,GENERAL ELECTRIC CO,8131383,2011-01-31,Method and system for rapid modeling and verif...,A system for collecting frequency response dat...,0


---

__Exercise 5 - Interpreting a model's "topics"__

- Back to [Table of Contents](#Table-of-Contents)

Examine the other topic IDs, and see if the "topics" we identified make sense as groupings of social service agencies.

In [55]:
topic_num = 3
print(processed_keywords[topic_num])
df_patent_desc[df_patent_desc.topic_id == topic_num].head(10)

inform, compon, layer, materi, request, non, compris, appli, form, structur


,business_name,patent_number,patent_date,patent_title,patent_abstract,topic_id
3,EASTMAN KODAK CO,7985684,2011-01-07,Actuating transistor including reduced channel...,A method of actuating a semiconductor device i...,3
14,SEALED AIR CORP,8033081,2011-01-14,Packaging machine,A packaging apparatus wherein each product is ...,3
21,MICROSOFT CORP,8041738,2011-02-10,Strongly typed tags,"In one or more embodiments, a tag is provided ...",3
39,IGT,8078868,2011-02-16,Multi-party encryption systems and methods,A cryptographic communication system and metho...,3
53,AIR PRODUCTS AND CHEMICALS INC,8092709,2011-05-17,Stabilizers for the stabilization of unsaturat...,A stabilized composition consists essentially ...,3
74,MICROSOFT CORP,8112409,2011-06-10,Predicting future queries from log data,"A system, media, and method for selecting futu...",3
78,MEADWESTVACO CORP,8113416,2011-07-14,Hermetically sealed paperboard container with ...,A hermetically sealed paperboard container inc...,3
85,HONEYWELL INTERNATIONAL INC,8118989,2011-08-11,Methods of bonding pure rhenium to a substrate,Methods are provided for bonding pure rhenium ...,3
109,GENERAL ELECTRIC CO,8138291,2011-05-23,"Anion exchange polymers, methods for making an...",A novel anion exchange polymer is provided. A ...,3
114,LOCKHEED MARTIN CORP,8143548,2011-01-06,Clamp for mixed mail sorter,A clamp is designed for accepting a mail piece...,3


## Further Resources

- Back to [Table of Contents](#Table-of-Contents)

A great resource for NLP in Python is 
[Natural Language Processing with Python](https://www.amazon.com/Natural-Language-Processing-Python-Analyzing/dp/0596516495).

# Supervised Learning: Document Classification

- Back to [Table of Contents](#Table-of-Contents)

Previously, we used topic modeling to infer relationships between social service facilities within the data. That is an example of unsupervised learning: we were looking to uncover structure in the form of topics, or groups of agencies, but we did not necessarily know the ground truth of how many groups we should find or which agencies belonged in which group.  

Now we turn our attention to supervised learning. In supervised learning, we have a *known* outcome or label (*Y*) that we want to produce given some data (*X*), and in general, we want to be able to produce this *Y* when we *don't* know it, or when we *only* have *X*. 

In order to produce labels we need to first have examples our algorithm can learn from, a "training set." In the context of text analysis, developing a training set can be very expensive, as it can require a large amount of human labor or linguistic expertise. **Document classification** is an example of supervised learning in which want to characterize our documents based on their contents (*X*). A common example of document classification is spam e-mail detection. Another example of supervised learning in text analysis is *sentiment analysis*, where *X* is our documents and *Y* is the state of the author. This "state" is dependent on the question you're trying to answer, and can range from the author being happy or unhappy with a product to the author being politically conservative or liberal. Another example is *part-of-speech tagging* where *X* are individual words and *Y* is the part-of-speech. 

In this section, we'll train a classifier to classify social service agencies. Let's see if we can label a new website as belonging to facility type "income" or "health."

## Supervised Learning - Prepare the Data

- Back to [Table of Contents](#Table-of-Contents)

In [193]:
# look at counts
df_patent_desc.factype.value_counts()

AttributeError: 'DataFrame' object has no attribute 'factype'

In [ ]:
# make a mask column we can use to flag rows with facility type in our types of interest.
mask = df_patent_desc.factype.isin(['income','health'])

In [ ]:
# use mask to subset our data.
df_income_health = df_patent_desc[mask]

Split into training and testing sets (20% held back for training):

In [ ]:
# split into train and test sets.
df_train, df_test = train_test_split(df_income_health, test_size=0.20, random_state=17)

Look at our training set:

In [ ]:
# look at our training set.
df_train.head()

In [ ]:
# make sure we only have the facility types we expect.
df_train['factype'].unique()

In [ ]:
# look at the counts for each value.
Counter(df_train['factype'].values)

Look at our testing set:

In [ ]:
# look at our testing set.
df_test.head()

In [ ]:
# make sure we only have the facility types we expect.
df_test['factype'].unique()

In [ ]:
# look at the counts for each value.
Counter(df_test['factype'].values)

## Prepare Data for Document Classification

- Back to [Table of Contents](#Table-of-Contents)

In order to feed out data into a classifier, we need to pull out the labels (*Y*) and a clean corpus of documents (*X*) for our training and testing sets. 

In [ ]:
# prepare training data - get labels we'll train on.
train_labels = df_train.factype.values

# prepare training data - clean text.
train_corpus = np.array( [re.sub(RE_PREPROCESS, ' ', text).lower() for text in df_train.textfromurl.values])

# prepare testing data - get labels we'll train on.
test_labels = df_test.factype.values

# prepare testing data - clean text.
test_corpus = np.array( [re.sub(RE_PREPROCESS, ' ', text).lower() for text in df_test.textfromurl.values])

# make list of all labels across train and test (should just be 'income' and 'health')
labels = np.append(train_labels, test_labels)

Just as we had done in the unsupervised learning context, we have to transform our data. This time we have to transform our testing and training set into two different bags of words. The classifier will learn from the training set, and we will evaluate the classifier's performance on the testing set.

First, we create a CountVectorizer that we'll use to convert our text documents to matrices of features based on words contained within our corpus.

In [ ]:
#parameters for vectorizer 
ANALYZER = "word" #unit of features are single words rather then phrases of words 
STRIP_ACCENTS = 'unicode'
TOKENIZER = None
NGRAM_RANGE = (0,2) #Range for pharases of words
MIN_DF = 0.01 # Exclude words that have a frequency less than the threshold
MAX_DF = 0.8  # Exclude words that have a frequency greater then the threshold 

vectorizer = CountVectorizer( analyzer = ANALYZER,
                              tokenizer = None, # alternatively tokenize_and_stem but it will be slower 
                              ngram_range = NGRAM_RANGE,
                              stop_words = stopwords.words( 'english' ),
                              strip_accents = STRIP_ACCENTS,
                              min_df = MIN_DF,
                              max_df = MAX_DF )

Next, we create a TF-IDF transformer, and create our bags of words, then weight them using TF-IDF.

In [ ]:
NORM = None          # turn on normalization flag
SMOOTH_IDF = True    # prevents division by zero errors
SUBLINEAR_IDF = True # replace TF with 1 + log(TF)
USE_IDF = True       # flag to control whether to use TFIDF

transformer = TfidfTransformer( norm = NORM,
                                smooth_idf = SMOOTH_IDF,
                                sublinear_tf = True )

# timing code - start!
start_time = time.time()

# get the bag-of-words for train and test from the vectorizer and
# then use TFIDF to limit the tokens found throughout the text 
train_bag_of_words = vectorizer.fit_transform( train_corpus ) #using all the data on for generating features!! Bad!
test_bag_of_words = vectorizer.transform( test_corpus )

# if we use IDF, compute it here.
if USE_IDF:
    train_tfidf = transformer.fit_transform(train_bag_of_words)
    test_tfidf = transformer.transform(test_bag_of_words)

# Get list of the feature names, for passing to our model.
features = vectorizer.get_feature_names()

# timing code - done!
print('Time Elapsed: {0:.2f}s'.format( time.time() - start_time ) )

We cannot pass the labels "income" or "health" directly to the classifier. Instead, we to encode them as 0s and 1s using the `labelencoder` part of `sklearn`. 

In [ ]:
#relabel our labels as a 0 or 1
le = preprocessing.LabelEncoder() 
le.fit(labels)
labels_binary = le.transform(labels)

In [ ]:
list(zip(labels,labels_binary))

We also need to create arrays of indices so we can access the training and testing sets accordingly.

In [ ]:
train_size = df_train.shape[ 0 ]
train_set_idx = np.arange( 0, train_size )
test_set_idx = np.arange( train_size, len( labels ) )
train_labels_binary = labels_binary[ train_set_idx ]
test_labels_binary = labels_binary[ test_set_idx ]

## Model Training - Train Document Classification Model

- Back to [Table of Contents](#Table-of-Contents)

The classifier we are using in the example is LogisticRegression. As we saw in the Machine Learning tutorial, first we decide on a classifier, then we fit the classifier to the data to create a model. We can then test our model on the test set by passing the features (*X*) from our test set to get predicted labels. The model will output the probability of each document being classified as income or health. 

In [ ]:
# create our LogisticRegression classifier.
clf = LogisticRegression(penalty='l1')

# train the classifer to create our model.
mdl = clf.fit( train_tfidf, labels_binary[ train_set_idx ] )

# create scores for each of the documents predicting whether each refers to 
#     an income or health agency
y_score = mdl.predict_proba( test_tfidf )

## Model Evaluation - Precision and Recall

- Back to [Table of Contents](#Table-of-Contents)

Now that we have calculated a score for each of our facility types of interest, we look at how well our model performed by outputting precision and recall curves at different cutoffs.

First, we define the function that will do this work:

In [ ]:
def plot_precision_recall_n( y_true, y_prob, model_name ):

    """
    y_true: ls
        ls of ground truth labels
    y_prob: ls
        ls of predic proba from model
    model_name: str
        str of model name (e.g, LR_123)
    """
    
    from sklearn.metrics import precision_recall_curve
    
    y_score = y_prob
    precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_true, y_score)
    precision_curve = precision_curve[:-1]
    recall_curve = recall_curve[:-1]
    pct_above_per_thresh = []
    number_scored = len(y_score)
    for value in pr_thresholds:
        num_above_thresh = len(y_score[y_score>=value])
        pct_above_thresh = num_above_thresh / float(number_scored)
        pct_above_per_thresh.append(pct_above_thresh)
    pct_above_per_thresh = np.array(pct_above_per_thresh)
    plt.clf()
    fig, ax1 = plt.subplots()
    ax1.plot(pct_above_per_thresh, precision_curve, 'b')
    ax1.set_xlabel('percent of population')
    ax1.set_ylabel('precision', color='b')
    ax1.set_ylim(0,1.05)
    ax2 = ax1.twinx()
    ax2.plot(pct_above_per_thresh, recall_curve, 'r')
    ax2.set_ylabel('recall', color='r')
    ax2.set_ylim(0,1.05)
    
    name = model_name
    plt.title(name)
    plt.show()

Then we output the graphs for our model:

In [ ]:
 plot_precision_recall_n( labels_binary[ test_set_idx ], y_score[:,1], 'LR' )

If we examine our precision-recall curve we can see that our precision is 1 up to 25 percent of the population. We can use a "precision at *k*" curve to see what percent of the corpus can be tagged by the classifier, and which should undergo a manual clerical review. Based on this curve, we might say that we can use our classifier to tag the 25% of the documents that have the highest scores as 1, and manually review the rest. 

Alternatively, we can try to maximize the entire precision-recall space. In this case we need a different metric - "Area Under Curve" (AUC). 

In [ ]:
def plot_precision_recall(y_true,y_score):
    """
    Plot a precision recall curve
    
    Parameters
    ----------
    y_true: ls
        ground truth labels
    y_score: ls
        score output from model
    """
    precision_curve, recall_curve, pr_thresholds = precision_recall_curve(y_true,y_score[:,1])
    plt.plot(recall_curve, precision_curve)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    auc_val = auc(recall_curve,precision_curve)
    print('AUC-PR: {0:1f}'.format(auc_val))
    plt.show()
    plt.clf()

In [ ]:
plot_precision_recall(labels_binary[test_set_idx],y_score)

 The AUC shows how accurate our scores are under different cutoff thresholds. The model will output a score between 0 and 1. We specify a  range of cutoff values and label all of the examples as 0 or 1 based on whether they are above or below each cutoff value. The closer our scores are to the true values, the more resilient they are to different cutoffs. For instance, if our scores were perfect, our AUC would be 1. 

## Model Evaluation - Feature Importances

- Back to [Table of Contents](#Table-of-Contents)

Next, we look at the importance of different features (words) in our model.

The function that will calculate these:

In [ ]:
def display_feature_importances( coef, features, labels, num_features = 10 ):

    """
    output feature importances
    
    Parameters
    ----------
    coef: numpy
        feature importances
    features: ls 
        feature names
    labels: ls
        labels for the classifier
    num_features: int
        number of features to output (default 10)
    
    Example
    --------
    
    
    """
    coef = mdl.coef_.ravel()

    dict_feature_importances = dict( zip(features, coef) )
    orddict_feature_importances = OrderedDict( 
                                    sorted(dict_feature_importances.items(), key=lambda x: x[1]) )

    ls_sorted_features  = list(orddict_feature_importances.keys())

    label0_features = ls_sorted_features[:num_features] 
    label1_features = ls_sorted_features[-num_features:] 

    print(labels[0],label0_features)
    print(labels[1], label1_features)

In [ ]:
display_feature_importances(mdl.coef_.ravel(), features, ['health','income'])

The feature importances give us the words which are the most relevant for distinguishing the type of social service agency (between income and health). Some of these make sense ("city church" seems more likely to be health than income), but some don't make as much sense, or seem to be artifacts from the website that we should remove ("housing humancarelogo"). 

--- 

### Exercise 6 - interpreting feature importances

- Back to [Table of Contents](#Table-of-Contents)

Display the top 25 feature importances to get an intution of which words are the most and least important. 

We need to know how to pass into the function we want the top 25 feature importances. We can do this by consulting the docstring of the function. 

From this docstring we can see that `num_features` is a keyword argument that is set to 10 by default. We can pass `num_features=25` into the keyword argument instead to get the top 25 feature importances. 

In [ ]:
display_feature_importances(mdl.coef_.ravel(), 
                            features,
                            ['health','income'],
                            num_features=25)

---

## Model Evaluation - Cross-validation

- Back to [Table of Contents](#Table-of-Contents)

Recall from the machine learning tutorial that we are seeking the find the most general pattern in the data in order to have to most general model that will be successful at classifying new unseen data. Our previous strategy above was the *Out-of-sample and holdout set*. With this strategy we try to find a general pattern by randomly dividing our data into a test and training set based on some percentage split (e.g., 50-50 or 80-20). We train on the test set and evaluate on the test set, where we pretend that we don't have the labels for the test set. A significant drawback with this approach is that we may be lucky or unlucky with our random split, and so our estimate of how we'd perform on truly new data is overly optimistic or overly pessimistic. A possible solution is to create many random splits into training and testing sets and evaluate each split to estimate the performance of a given model. 

A more sophisticated holdout training and testing procedure is *cross-validation*. In cross-validation we split our data into *k* folds or partitions, where *k* is usually 5 or 10. We then iterate k times. In each iteration, one of the folds is used as a test set, and the rest of the folds are combined to form the training set. We can then evaluate the performance at each iteration to estimate the performance of a given method. An advantage of using cross-validation is all examples of data are used in the training set at least once. 

Define function to create test and train bags of words:

In [ ]:
def create_test_train_bag_of_words(train_corpus, test_corpus):

    """
    Create test and training set bag of words
    
    
    Parameters
    ----------
    train_corpus: ls
        ls of raw text for text corpus.
    test_corpus: ls
        ls of raw text for train corpus. 
        
    Returns
    -------
    (train_bag_of_words,test_bag_of_words): scipy sparse matrix
        bag-of-words representation of train and test corpus
    features: ls
        ls of words used as features. 
    """

    #parameters for vectorizer 
    ANALYZER = "word" #unit of features are single words rather then phrases of words 
    STRIP_ACCENTS = 'unicode'
    TOKENIZER = None
    NGRAM_RANGE = (0,2) #Range for pharases of words
    MIN_DF = 0.01 # Exclude words that have a frequency less than the threshold
    MAX_DF = 0.8  # Exclude words that have a frequency greater then the threshold 

    vectorizer = CountVectorizer(analyzer=ANALYZER,
                                tokenizer=None, # alternatively tokenize_and_stem but it will be slower 
                                ngram_range=NGRAM_RANGE,
                                stop_words = stopwords.words('english'),
                                strip_accents=STRIP_ACCENTS,
                                min_df = MIN_DF,
                                max_df = MAX_DF)
    
    NORM = None #turn on normalization flag
    SMOOTH_IDF = True #prevents division by zero errors
    SUBLINEAR_IDF = True #replace TF with 1 + log(TF)
    USE_IDF = True #flag to control whether to use TFIDF

    transformer = TfidfTransformer(norm = NORM,smooth_idf = SMOOTH_IDF,sublinear_tf = True)

    #get the bag-of-words from the vectorizer and
    #then use TFIDF to limit the tokens found throughout the text 
    train_bag_of_words = vectorizer.fit_transform( train_corpus ) 
    test_bag_of_words = vectorizer.transform( test_corpus )
    if USE_IDF:
        train_tfidf = transformer.fit_transform(train_bag_of_words)
        test_tfidf = transformer.transform(test_bag_of_words)
    features = vectorizer.get_feature_names()

    
    return train_tfidf, test_tfidf, features

And now, use scikit-learn's StratifiedKFold object to generate our folds (we'll do 3 here as an example), and then train and validate across all combinations.

In [ ]:
from sklearn.cross_validation import StratifiedKFold

# create folds
cv = StratifiedKFold( train_labels_binary, n_folds = 3 )

# get our labels
train_labels_binary = le.transform(train_labels)

# for each fold, get rows specified as train and test, then
#     train and test our model with it.
for i, ( train, test ) in enumerate(cv):
    
    # break out train and test data.
    cv_train = train_corpus[train]
    cv_test = train_corpus[test]
    
    # create bags of words and get feature names
    bag_of_words_train, bag_of_words_test, feature_names = create_test_train_bag_of_words(cv_train, 
                                                                                          cv_test)
    # fit our model and then use it to predict values in test set.
    probas_ = clf.fit(bag_of_words_train, 
                      train_labels_binary[train]).predict_proba(bag_of_words_test)
    cv_test_labels = train_labels_binary[test]
    
    # draw precision and recall curve
    precision_curve, recall_curve, pr_thresholds = precision_recall_curve(cv_test_labels, probas_[:,1] )

    # calculate and plot AUC
    auc_val = auc( recall_curve,precision_curve )
    plt.plot( recall_curve, precision_curve, label = 'AUC-PR {0} {1:.2f}'.format( i, auc_val ) )

#-- END loop over folds --#
                                                                          
# and, plot the collected graphs.
plt.ylim( 0, 1.05 )    
plt.xlabel( 'Recall' )
plt.ylabel( 'Precision' )
plt.legend( loc = "lower left", fontsize = 'x-small' )

In this case we did 3-fold cross-validation and plotted precision-recall curves for each iteration. You can see that there is a marked difference between the iterations. We can then average the AUC-PR of each iteration to estimate the performance of our method. 

---
### Exercise 7 - Try a 5-fold cross-validation

- Back to [Table of Contents](#Table-of-Contents)

Try 5-fold cross-validation. 

In [ ]:
from sklearn.cross_validation import StratifiedKFold
cv = StratifiedKFold(train_labels_binary, n_folds=5)
train_labels_binary = le.transform(train_labels)
for i, (train,test) in enumerate(cv):
    cv_train = train_corpus[train]
    cv_test = train_corpus[test]
    bag_of_words_train, bag_of_words_test, feature_names = create_test_train_bag_of_words(cv_train, 
                                                                                          cv_test)
    
    probas_ = clf.fit(bag_of_words_train, 
                      train_labels_binary[train]).predict_proba(bag_of_words_test)
    cv_test_labels = train_labels_binary[test]
    
    precision_curve, recall_curve, pr_thresholds = precision_recall_curve(cv_test_labels,
                                                                          probas_[:,1])
    auc_val = auc(recall_curve,precision_curve)
    plt.plot(recall_curve, precision_curve, label='AUC-PR {0} {1:.2f}'.format(i,auc_val))
    
plt.ylim(0,1.05)    
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(loc="lower left", fontsize='x-small')

---

## Model Output - Examples of Document Classification

- Back to [Table of Contents](#Table-of-Contents)

Look at our data:

In [ ]:
df_test

And, look at the text the model is most certain of being categorized as each facility type:

In [ ]:
num_comments = 2
label0_comment_idx = y_score[:,1].argsort()[:num_comments] 
label1_comment_idx = y_score[:,1].argsort()[-num_comments:]
test_set_labels = labels[test_set_idx]
#convert back to the indices of the original dataset
top_comments_testing_set_idx = np.concatenate([label0_comment_idx, 
                                               label1_comment_idx])


#these are the 5 comments the model is most sure of 
for i in top_comments_testing_set_idx:
    print(
        u"""{}:{}\n---\n{}\n===""".format(test_set_labels[i],
                                          y_score[i,1],
                                          test_corpus[i]))

These are the top 2 examples that the model is the most sure of for each label. We can see our important feature words in the descriptions, which gives a hint of how the model made these classifications. 